In [1]:
import sys
from datasets import load_from_disk
from fastNLP import DataSet,Vocabulary,prepare_torch_dataloader
from fastNLP.io import DataBundle

sys.path.append('..')
nerdata = load_from_disk(dataset_path='../peoples_daily_ner')
train_dataset = DataSet.from_pandas(nerdata['train'].to_pandas())
val_dataset = DataSet.from_pandas(nerdata['validation'].to_pandas())
test_dataset = DataSet.from_pandas(nerdata['test'].to_pandas())


datasets = {"train":train_dataset,"val":val_dataset,"test":test_dataset}
data_bundle = DataBundle(datasets=datasets)

/remote-home/pjli/anaconda3/envs/nlptorch/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data_bundle.apply_more(lambda ins:{'words': ins['tokens'], 'seq_len': len(ins['tokens']), 'target': ins['ner_tags']},
                   progress_bar="tqdm")
data_bundle.delete_field('tokens')
data_bundle.delete_field('ner_tags')
data_bundle.delete_field('id')
data_bundle.drop(lambda ins:ins['seq_len'] <= 0) # 清理长度为0 的数据，否则构建词典出问题

In total 3 datasets:
	train has 20864 instances.
	val has 2318 instances.
	test has 4636 instances.

In [3]:
token_vocab = Vocabulary()
token_vocab.from_dataset(data_bundle.get_dataset('train'), field_name='words')
token_vocab.index_dataset(data_bundle.get_dataset('train'), field_name='words')
token_vocab.index_dataset(data_bundle.get_dataset('val'), field_name='words')
token_vocab.index_dataset(data_bundle.get_dataset('test'), field_name='words')
label_vocab = Vocabulary(padding=None, unknown=None)
label_vocab.add_word_lst(['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC'])

Vocabulary(['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG']...)

In [4]:
train_dataloader = prepare_torch_dataloader(train_dataset, batch_size=16, shuffle=True)
evaluate_dataloader = prepare_torch_dataloader(val_dataset, batch_size=16)
test_dataloader = prepare_torch_dataloader(test_dataset,batch_size=16)

In [5]:
from fastNLP.models.torch import BiLSTMCRF
from fastNLP import Trainer, Evaluator, SpanFPreRecMetric,LoadBestModelCallback, TorchWarmupCallback, Accuracy

callbacks = [
    LoadBestModelCallback(),
    TorchWarmupCallback(),
]

model = BiLSTMCRF(embed=(len(token_vocab), 150), num_classes=len(label_vocab),
                  num_layers=1, hidden_size=150, dropout=0.2)

from torch.optim import AdamW

optimizers = AdamW(params=model.parameters(), lr=1e-3)

In [7]:
from FGM import FGM
from PGD import PGD
from FreeLB import FreeLB

fgm = FGM(model)
pgd = PGD(model)
freelb = FreeLB(model)

def batch_step_fn_fgm(trainer, batch):
    outputs = trainer.train_step(batch)
    trainer.backward(outputs)
    # 对抗训练
    fgm.attack()
    outputs = trainer.train_step(batch)
    trainer.backward(outputs)
    fgm.restore()
    trainer.step()
    trainer.zero_grad()

def batch_step_fn_pgd(trainer, batch):
    outputs = trainer.train_step(batch)
    trainer.backward(outputs)
    pgd.backup_grad()
    K = pgd.k
    for t in range(K):
        pgd.attack(is_first_attack=(t==0))
        if t == K-1:
            pgd.restore_grad()
        else:
            trainer.zero_grad()
            
        outputs = trainer.train_step(batch)
        trainer.backward(outputs)
    pgd.restore()
    trainer.step()
    trainer.zero_grad()
    
def batch_step_fn_freelb(trainer, batch):
    outputs = trainer.train_step(batch)
    trainer.backwards(outputs)
    K = freelb.k
    for t in range(K):
        freelb.backup_grad()
        trainer.zero_grad()
        freelb.attack(is_first_attack=(t==0))        
        outputs = trainer.train_step(batch)
        trainer.backward(outputs)
        freelb.backup_r_grad()
        freelb.upgrade_grad()
        freelb.upgrade_r_at()
    freelb.restore()
    trainer.step()
    trainer.zero_grad()

In [8]:
deviceId = 7
batch_step_fn = batch_step_fn_pgd
trainer = Trainer(
    model=model,
    driver='torch',
    device=deviceId,  # 'cuda'
    n_epochs=10,
    optimizers=optimizers,
    train_dataloader=train_dataloader,
    evaluate_dataloaders=evaluate_dataloader,
    metrics={'F1': SpanFPreRecMetric(tag_vocab=label_vocab),'acc':Accuracy()},
    progress_bar = 'tqdm',
    evaluate_every= 200, # -1代表每个epoch一次评估,正数代表batch个批次一次评估
    callbacks=callbacks,
    monitor='f#F1',
    batch_step_fn=batch_step_fn,
    
)

testor = Evaluator(
    model=model,
    driver='torch',
    device = deviceId,
    dataloaders=test_dataloader,
    metrics={'F1': SpanFPreRecMetric(tag_vocab=label_vocab),'acc':Accuracy()}
) 

In [9]:
trainer.run()

[15:55:59] INFO     Running evaluator sanity check for 2 batches.              ]8;id=880304;file:///remote-home/pjli/anaconda3/envs/nlptorch/lib/python3.7/site-packages/fastNLP/core/controllers/trainer.py\trainer.py]8;;\:]8;id=974825;file:///remote-home/pjli/anaconda3/envs/nlptorch/lib/python3.7/site-packages/fastNLP/core/controllers/trainer.py#661\661]8;;\

Epoch:0:   2%|▏         | [00:51<1:17:42, 473.47s/it, ]   



















































[15:56:58] INFO     ++++++++++Eval. results on Epoch:0,              ]8;id=465354;file:///remote-home/pjli/anaconda3/envs/nlptorch/lib/python3.7/site-packages/fastNLP/core/callbacks/progress_callback.py\progress_callback.py]8;;\:]8;id=606710;file:///remote-home/pjli/anaconda3/envs/nlptorch/lib/python3.7/site-packages/fastNLP/core/callbacks/progress_callback.py#322\322]8;;\
                    Batch:200++++++++++                                                      

           INFO     {"f#F1": 0.0, "pre#F1": 0.0, "rec#F1": 0.0,      ]8;id=229299;file:///remote-home/pjli/anaconda3/envs/nlptorch/lib/python3.7/site-packages/fastNLP/core/callbacks/progress_callback.py\progress_callback.py]8;;\:]8;id=467902;file:///remote-home/pjli/anaconda3/envs/nlptorch/lib/python3.7/site-packages/fastNLP/core/callbacks/progress_callback.py#327\327]8;;\
                    "acc#acc": 0.859152}                                                     

Epoch:0:   3%|▎         | [01:46<55:08, 341.32s/it, ]   










































...

[16:57:59] INFO     ----------Eval. results on Epoch:9,              ]8;id=63901;file:///remote-home/pjli/anaconda3/envs/nlptorch/lib/python3.7/site-packages/fastNLP/core/callbacks/progress_callback.py\progress_callback.py]8;;\:]8;id=276814;file:///remote-home/pjli/anaconda3/envs/nlptorch/lib/python3.7/site-packages/fastNLP/core/callbacks/progress_callback.py#322\322]8;;\
                    Batch:1264----------                                                     

           INFO     {"f#F1": 0.833759, "pre#F1": 0.856433, "rec#F1": ]8;id=357201;file:///remote-home/pjli/anaconda3/envs/nlptorch/lib/python3.7/site-packages/fastNLP/core/callbacks/progress_callback.py\progress_callback.py]8;;\:]8;id=965326;file:///remote-home/pjli/anaconda3/envs/nlptorch/lib/python3.7/site-packages/fastNLP/core/callbacks/progress_callback.py#327\327]8;;\
                    0.812255, "acc#acc": 0.978183}                                           

Epoch:9: 100%|█████████▉| [1:02:11<00:00, 341.95s/it, ] 

[16:58:11] INFO     The best performance for monitor f#F1:0.836074    ]8;id=868052;file:///remote-home/pjli/anaconda3/envs/nlptorch/lib/python3.7/site-packages/fastNLP/core/callbacks/progress_callback.py\progress_callback.py]8;;\:]8;id=459827;file:///remote-home/pjli/anaconda3/envs/nlptorch/lib/python3.7/site-packages/fastNLP/core/callbacks/progress_callback.py#37\37]8;;\
                    was achieved in Epoch:9, Global Batch:12600. The                         
                    evaluation result:                                                       
                    {'f#F1': 0.836074, 'pre#F1': 0.861628, 'rec#F1':                         
                    0.811993, 'acc#acc': 0.978338}                                           

           INFO     Loading best model from buffer with f#F1: ]8;id=87438;file:///remote-home/pjli/anaconda3/envs/nlptorch/lib/python3.7/site-packages/fastNLP/core/callbacks/load_best_model_callback.py\load_best_model_callback.py]8;;\:]8;id=45455;file:///remote-home/pjli/anaconda3/envs/nlptorch/lib/python3.7/site-packages/fastNLP/core/callbacks/load_best_model_callback.py#120\120]8;;\
                    0.836074 (achieved in Epoch: 9, Global                                   
                    Batch: 12600) ...                                                        

In [10]:
trainer.evaluator.run()

{'f#F1': 0.836074, 'pre#F1': 0.861628, 'rec#F1': 0.811993, 'acc#acc': 0.978338}

In [11]:
testor.run()

/remote-home/pjli/anaconda3/envs/nlptorch/lib/python3.7/site-packages/rich/live.py:221: 
UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

{'f#F1': 0.837955, 'pre#F1': 0.859384, 'rec#F1': 0.817568, 'acc#acc': 0.978613}

{'f#F1': 0.837955, 'pre#F1': 0.859384, 'rec#F1': 0.817568, 'acc#acc': 0.978613}

In [12]:
import os
 
pid = os.getpid()
!kill -9 $pid

: 

: 